<a href="https://colab.research.google.com/github/NickKornienko/Language-Identification-model/blob/main/LanguageIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [2]:
%pip install librosa
%pip install tensorflow
%pip install jupyter

# %pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Ensure GPU is being used


In [13]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.test.gpu_device_name():
    print('Default GPU Device:', tf.test.gpu_device_name())
else:
    print("Please install GPU version of TF")

Num GPUs Available:  0
Please install GPU version of TF


Feature extraction


In [16]:
import librosa
import numpy as np


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None

    return mfccs_processed

Load data


In [17]:
# Path to your audio files
audio_path = "audio_files"
languages = ['english', 'spanish', 'french']  # List of languages

# Variables to hold features and labels
features = []
labels = []

# Loop through each language and each file
for language in languages:
    audios = [f for f in os.listdir(
        audio_path + '/' + language) if f.endswith('.wav')]
    for file in audios:
        feature = extract_features(audio_path + '/' + language + '/' + file)
        if feature is not None:
            features.append(feature)
            labels.append(language)

# Convert features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode the labels into integers
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
train_x, test_x, train_y, test_y = train_test_split(
    features, label_encoded, test_size=0.2, random_state=42)

NameError: name 'os' is not defined

Model building


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

num_labels = 3  # English, Spanish, French

model = Sequential()
model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

Complie, train, fit then save the model


In [18]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'], optimizer='adam')

# Fit the model
model.fit(train_x, train_y, batch_size=32, epochs=50,
          validation_data=(test_x, test_y))

# Save the entire model to a HDF5 file
model.save('my_language_model.h5')

print("Model saved to disk as my_language_model.h5")

NameError: name 'train_y' is not defined

Load the model


In [ ]:
from tensorflow.keras.models import load_model

# Load the entire model back
loaded_model = load_model('my_language_model.h5')

print("Model loaded from disk.")

Check model accuracy


In [ ]:
score = model.evaluate(test_x, test_y)
print("Accuracy: ", score[1])